#  강화학습을 이용한 인공지능 슈퍼마리오 만들기 메뉴얼

### 이 메뉴얼은 슈퍼마리오 환경 설치부터 강화학습 알고리즘(DQN)을 이용해 똑똑한 '슈퍼'마리오를 만들기 위한 메뉴얼 입니다

# Let's make my OWN! A.I. SuperMario

In [1]:

import gym
import ppaquette_gym_super_mario
import random

import numpy as np
import gym 
from wrapper import action_space
from keras.layers.convolutional import Conv2D
from keras.layers import Dense, Flatten, Input, Lambda, merge
from keras.optimizers import RMSprop
from keras.models import Model, Sequential
#from skimage.transform import resize
from skimage.color import rgb2gray
from collections import deque
from keras import backend as K
import tensorflow as tf
import numpy as np

from sum_tree import SumTree
import random


load_model = True

epsilon = 0.1
epsilon_start, epsilon_end = 0.1, 0.1
exploration_steps = 400000.

epsilon_decay_step = (epsilon_start - epsilon_end) \
                                  / exploration_steps
batch_size = 32

discount_factor = 0.99

train_start= 10000
update_target_rate = 10000
memory=deque(maxlen=1000000)
just_start_train=True
state_size = (84,84,4)
action_size =6
env = gym.make('ppaquette/meta-SuperMarioBros-v0')

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Deep learning model

In [2]:
env.observation_space

Box(224, 256, 3)

In [3]:
def build_model():
    model = Sequential()
    model.add(Conv2D(32,(8,8),strides =(4,4), activation='relu', input_shape=state_size))
    model.add(Conv2D(64,(4,4), strides=(2,2), activation='relu'))
    model.add(Conv2D(64,(3,3), strides =(1,1), activation = 'relu'))
    model.add(Flatten())
    model.add(Dense(512,activation='relu'))
    model.add(Dense(action_size))
    model.summary()

    return model

# Define model and target model

In [4]:
model = build_model()
target_model=build_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 20, 20, 32)        8224      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 64)          32832     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               1606144   
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 3078      
Total params: 1,687,206
Trainable params: 1,687,206
Non-trainable params: 0
_________________________________________________________________


### append <s,a,r,s'> at replay memory

# Functions 

### change action(number) to the action( action what I think I need )

### append <s,a,r,s'> at replay memory

In [5]:
import random
action_size=6
epsilon = 0.1


In [6]:
#making an greedy action

action_size=6
def get_action(history):
        history = np.float32(history / 255.0)
        if np.random.rand() <= epsilon:
            return random.randrange(action_size)
        else:
            q_value = model.predict(history)
            return np.argmax(q_value[0])

In [7]:
def append_sample(history, action, reward, next_history, dead):
    memory.append((history,action,reward,next_history,dead))

### Optimizer

In [8]:
def optimizer(self):
        a = K.placeholder(shape=(None,), dtype='int32')
        y = K.placeholder(shape=(None,), dtype='float32')

        prediction = model.output

        a_one_hot = K.one_hot(a, action_size)
        q_value = K.sum(prediction * a_one_hot, axis=1)
        error = K.abs(y - q_value)

        quadratic_part = K.clip(error, 0.0, 1.0)
        linear_part = error - quadratic_part
        loss = K.mean(0.5 * K.square(quadratic_part) + linear_part)

        optimizer = RMSprop(lr=0.00025, epsilon=0.01)
        updates = optimizer.get_updates(model.trainable_weights, [], loss)
        train = K.function([model.input, a, y], [loss], updates=updates)

        return train

### Update models 


In [9]:
def update_target_model():
        target_model.set_weights(model.get_weights())

### Load Model

In [10]:
def load_model(filename):
    model.load_weights(filename)

## Train model
(Batch from the memory)

In [11]:


def train_model():
    global epsilon
    if epsilon > epsilon_end:
        epsilon -= epsilon_decay_step
    
    mini_batch= random.sample(memory, batch_size)
    #mini_batch = memory.sample(batch_size)
    
  
    history = np.zeros((batch_size, state_size[0],
                            state_size[1], state_size[2]))
    next_history = np.zeros((batch_size, state_size[0],
                             state_size[1], state_size[2]))
    target = np.zeros((batch_size,))
    action, reward, dead = [], [], []

    for i in range(batch_size):
            history[i] = np.float32(mini_batch[i][0] / 255.)
            next_history[i] = np.float32(mini_batch[i][3] / 255.)
            action.append(mini_batch[i][1])
            reward.append(mini_batch[i][2])
            dead.append(mini_batch[i][4])

    target_value = target_model.predict(next_history)

    for i in range(batch_size):
        if dead[i]:
            target[i] = reward[i]
        else:
            target[i] = reward[i] + discount_factor * \
                                        np.amax(target_value[i])

    loss = optimizer([history, action, target])

    
    
    

### LET's make

In [12]:

    
    

wra_act=action_space
scores, episodes, global_step = [], [], 0

#reduce actions
env=wra_act.mario_action(env)

#reduce inputsize to 84,84,1
env=wra_act.ProcessFrame84(env)


obs=env.reset()
model = build_model()

load_model("./save_ws/supermario_again_ws_v6.h5")


for e in range(10000):
        done = False
        
        step, score, start_life = 0, 0, 5
        
        i=0
        i+=1
        
        reshape_obs=np.reshape([obs],(1,84,84,1))
        history=np.stack((obs,obs,obs,obs), axis = 2)
        history = np.reshape([history], (1, 84, 84, 4))
      #  history=np.append(reshape_obs, history[:,:,:,:3], axis=3)
        #check
       # history=np.reshape([history],(84,84,4))
        
        action=[0,0,0,1,0,0]
        action2=[0,0,0,1,1,0]
    

        while not done:

  

            # 0: stop, 3: left, 4: left jump, 7:right, 8:right jump 11: jump
            action=get_action(history)
            if action==0:
                actions=[0,0,0,1,0,0]
            elif action==2:
                actions=[0,0,0,1,1,0]
    
            elif action==1:
                actions=[0,0,0,1,1,0]
            elif action==3:
                actions=[0,0,0,1,1,0]
            elif action==4:
                actions=[0,0,0,1,0,0]
    
            elif action==5:
                actions=[0,0,0,1,1,0]
            elif action==6:
                actions=[0,0,0,0,1,0]
    
           
            # 선택한 행동으로 환경에서 한 타임스텝 진행
            observe, reward, done, clear= env.step(actions)
            
            if clear:
                reward += 30
                done = True
            distance=clear.get('distance')
            if distance== 722:
                print('here')
            print(distance)
            if done and not clear:
                reward = -30
              
            reward = np.clip(reward, -1., 1.)
            next_state = observe
            
            next_state = np.reshape([next_state], (1, 84, 84, 1))
            next_history=np.stack((next_state ,next_state ,next_state ,next_state ), axis = 2)
            next_history= np.reshape([next_history], (1, 84, 84, 4))

         #   agent.avg_q_max += np.amax(
       #         agent.model.predict(np.float32(history / 255.))[0])

            
         #   append_sample(history, action, reward, next_history, done)

    


         #   history = next_history
            
 
         

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 20, 20, 32)        8224      
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 9, 9, 64)          32832     
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 7, 7, 64)          36928     
_________________________________________________________________
flatten_3 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               1606144   
_________________________________________________________________
dense_6 (Dense)              (None, 6)                 3078      
Total params: 1,687,206
Trainable params: 1,687,206
Non-trainable params: 0
_________________________________________________________________


OSError: Unable to open file (unable to open file: name = './save_ws/supermario_again_ws_v6.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
clear.dis